In [1]:
import numpy as np
n = 60000
import h5py
from load_cifar10_db import load_images_cifar,load_scattering_cifar
J=3
L=8
m=2

X_train, y_train, X_test, y_test = load_scattering_cifar(num_images = 150000, J=J,L=L,m=m, sigma_phi=0.8, sigma_xi=0.8)
#

#Load the saved data    
#f = h5py.File('./cifar10_scatteringdata.mat')
#X_train = np.array(f['X_train']).astype('single')
#y_train = np.array(f['y_train']).astype('single')
#X_test = np.array(f['X_test']).astype('single')
#y_test = np.array(f['y_test']).astype('single')
#m = np.array(f['m'])
#J = np.array(f['J'])
#L = np.array(f['L'])

num_samples = X_train.shape[0]

num_coeffs = 3*(1+J*L+J*L*L*(J-1)/2)

X_train.shape
Xtrain_1d = X_train.reshape((50000,num_coeffs*8*8))
Xtest_1d = X_test.reshape((10000,num_coeffs*8*8))


Loading images:
X_train shape: (150000, 32, 32)
150000 train samples
30000 test samples
150000  images loaded in  27.51226806640625  secs
shape data: (150000, 32, 32)
Create filters:
Compute  150000  scatterings:
0 / 150000
600 / 150000
1200 / 150000
1800 / 150000
2400 / 150000
3000 / 150000
3600 / 150000
4200 / 150000
4800 / 150000
5400 / 150000
6000 / 150000
6600 / 150000
7200 / 150000
7800 / 150000
8400 / 150000
9000 / 150000
9600 / 150000
10200 / 150000
10800 / 150000
11400 / 150000
12000 / 150000
12600 / 150000
13200 / 150000
13800 / 150000
14400 / 150000
15000 / 150000
15600 / 150000
16200 / 150000
16800 / 150000
17400 / 150000
18000 / 150000
18600 / 150000
19200 / 150000
19800 / 150000
20400 / 150000
21000 / 150000
21600 / 150000
22200 / 150000
22800 / 150000
23400 / 150000
24000 / 150000
24600 / 150000
25200 / 150000
25800 / 150000
26400 / 150000
27000 / 150000
27600 / 150000
28200 / 150000
28800 / 150000
29400 / 150000
30000 / 150000
30600 / 150000
31200 / 150000
31800 / 15000

/Users/ferradans/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/ferradans/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [2]:
# Save data in a .mat file
import h5py
m=1
J=3
L=8
with h5py.File('./cifar10_scatteringdata_sigma08_m2.mat','w') as hf:
    hf.create_dataset('X_train', data=X_train)
    hf.create_dataset('y_train', data=y_train)
    hf.create_dataset('X_test' , data = X_test)
    hf.create_dataset('y_test' , data = y_test)
    hf.create_dataset('m' , data = m)
    hf.create_dataset('J' , data = J)
    hf.create_dataset('L' , data = L)


In [3]:
import numpy as np
def gethomogeneus_datast(X,y,d,n):
    num_per_class = np.int(n/d)
    ytrain=np.reshape(y,(y.shape[0],))

    X_out = []#np.zeros(n,X.shape[1],X.shape[2],X.shape[3])
    y_out = []
    for i_d in np.arange(d):
        indx = np.where(ytrain.ravel()==i_d)

        X_out.append(X[indx[0][0:num_per_class],:])
        y_out.append(ytrain[indx[0][0:num_per_class],])

    
    X_out = np.concatenate(X_out,axis=0)
    y_out = np.concatenate(y_out,axis=0)
    #print(X_out.shape)
    #print(y_out)
    return X_out,y_out

In [7]:
## Testing specific values

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,y_train,10,n)    

bestC = 4.4
bestgamma = 10**-3.55 
print('C=',bestC)
print('gamma=',bestgamma)
gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),StandardScaler(),gs_gaussian)

pip_gaussian.fit(Xa,ya)
out=pip_gaussian.predict(Xtest_1d)
score = accuracy_score(y_test, out)
print(score) # C=4.4, log10(gamma) = -3.5, score=0.338



C= 4.4
gamma= 0.0002818382931264455
0.4482


In [ ]:
# Linear kernel

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,y_train,10,n)    

parameters = {'C':np.arange(0.00001,0.1,0.025)}
gs_gaussian = GridSearchCV(SVC(kernel='linear'),param_grid=parameters)

pip_gaussian = make_pipeline(StandardScaler(),gs_gaussian)
pip_gaussian.fit(Xa,ya)

C_best = gs_gaussian.best_params_['C']

out=pip_gaussian.predict(Xtest_1d)
score = accuracy_score(y_test, out)
print(score) # C=0.1, score(not 1-score )=0.4869
print('best C:',C_best)

In [ ]:
##### Gaussian kernel grid search
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import time

gammavec = 10**np.arange(-3.6,-3.4,0.1) #0.3-1.5(0.25) got 1.995 #(-1,0.5,0.1) got 0.5
gammavec.shape = (gammavec.shape[0])
Cvec = np.arange(3.8,4.5,0.2) #3-5(0.25))got 4.25  #4-5, 4.0 (0.5)
Cvec.shape=(Cvec.shape[0])
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf',tol=1,max_iter=-1),parameters)
pip_gaussian = make_pipeline(MinMaxScaler(),StandardScaler(),gs_gaussian)
n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,y_train,10,n)

start = time.time()
pip_gaussian.fit(Xa,ya.ravel())

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(gamma=np.log10(bestgamma))
print(C=bestC)
out=pip_gaussian.predict(Xtest_1d)

score = accuracy_score(y_test, out)
print(score) # gamma = -3.5, C=4.1, score=0.4179, tol = 0.1 , time: 45.95646786689758,n=300
print('time:',(time.time()-start)/60)

In [14]:
m=2, n=300
-3.5
3.8
0.3243
time: 408.93535590171814

n=40000
Using Theano backend.
8.0
num coefs: 75
X_train shape: (50000, 75, 8, 8)
Crossvalidation for  25  coefficients
Cross validation for  40000  samples done in  38865.804097890854  secs (25 options)
Best gamma: 0.000316227766017
Best C: 4.5




n=1000, max_iter = 75, tol = 0.1
-3.6
4.0
0.5091
time: 214.2964949607849

max_iter=-1, n=300
-3.6
3.8
0.4204
time: 35.526423931121826

max_iter = 100    
-3.6
3.8
0.4204
time: 31.296122074127197
max_iter = 50
-3.6
3.8
0.4213
time: 32.86830997467041
    

n=10000 tol = 0.1
-3.5
4.2
0.6671

n=5000, tol = 0.1
-3.6
4.0
0.6255



n=1000, tol = 0.1 (MinMaxScaler, Stdscaler)
-3.5
3.8
0.512


n=1000, tol = 0.1 (down: just standadScaler())
-3.6
3.8
score=0.5169
time: 387.4123179912567

tol = 0.01 
-3.6
3.8
0.5169
time: 461.48333406448364


________________________
n = 300, tol = 0.001
-3.5
4.1
0.4185
time: 46.558130979537964
    
-3.6
4.0
0.4204
time: 36.48983287811279
    
    

-3.5


In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer

gammavec = np.array(10**np.arange(0.46,0.52,0.01)) #np.array(10**np.arange(-0.1,1,0.3))  best 0.5 #np.array(10**np.arange(-3,-0.5,0.1)) got 0.0063095734448
gammavec.shape = (6)
Cvec = np.arange(3.8,3.95,0.05)
#Cvec.append(1) #np.arange(3.15,3.55,0.10) #np.arange(0.5,4,0.25) got 3.25
Cvec.shape=(4)
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(StandardScaler(),gs_gaussian)
pip_gaussian.fit(Xa,ya)

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestC)
print(bestgamma)
gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

pip_gaussian.fit(Xa,ya)
out=pip_gaussian.predict(Xtest_1d)
score = 1.0-accuracy_score(ytest, out)
print(score) # C=3, gamma = 3.16227766, score=0.0139
#C=3.9, gamma = 2.88403150313 score=0.0137

In [ ]:
from skimage.color.colorconv import rgb2yuv,yuv2rgb

Xta = X_train.transpose((3,2,0,1))/255
Iyuv=rgb2yuv(Xta)
Iyuv = Iyuv.transpose((2,3,0,1)).copy()
Iyuv.shape = (num_samples*3,32,32)

In [ ]:
im = Iyuv[0:3,:,:].transpose((1,2,0))                      
imvuelta = yuv2rgb(im)
imvuelta.max()

In [ ]:
import matplotlib.pylab as plt 
%matplotlib inline
indx = 0
plt.subplot(1,2,1)
plt.imshow(imvuelta)
plt.subplot(1,2,2)
x = X_train[indx,:,:,:]
x.shape
plt.imshow(x.transpose((2,1,0))/255)
plt.show()

In [ ]:
import matplotlib.pylab as plt 
%matplotlib inline
indx = 0
plt.subplot(1,2,1)
plt.imshow(Ivuelta[:,:,indx,:])
plt.subplot(1,2,2)
x = X_train[indx,:,:,:]
x.shape
plt.imshow(x.transpose((2,1,0))/255)
plt.show()

In [ ]:
n = 60000
from load_mnist_db import load_scattering_mnist

import time as time
import numpy as np
from keras.datasets import mnist
num_images = n
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_mnist(num_images = num_images, px=px,J=3,L=6,m=2)
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
num_samples,num_features = Xtrain_1d.shape

In [ ]:
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xtest_1d.shape

In [ ]:
"""
import os, sys
import scipy.misc
import scipy.io
print('saving data in ./mnist_scat_complet.mat')

scipy.io.savemat('./mnist_scat_complet.mat', 
                 mdict={'ytest': ytest, 'xtest': Xtest, 
                        'ytrain': ytrain, 'xtrain': Xtrain })
print('saving done!')
"""

In [ ]:
"""
import h5py
import numpy as np

f = h5py.File('mnist_scat_complet.mat')
Xtrain = np.array(f['xtrain']).transpose(3,2,1,0)
Xtest = np.array(f['xtest']).transpose(3,2,1,0)
ytrain = f['ytrain']
ytest = f['ytest']
"""

In [8]:
1000/24

41.666666666666664

In [ ]:
def gethomogeneus_datast(X,y,d,n):
    num_per_class = np.int(n/d)
    ytrain=np.reshape(y,(y.shape[0],))

    X_out = []#np.zeros(n,X.shape[1],X.shape[2],X.shape[3])
    y_out = []
    for i_d in np.arange(d):
        indx = np.where(ytrain.ravel()==i_d)

        X_out.append(X[indx[0][0:num_per_class],:])
        y_out.append(ytrain[indx[0][0:num_per_class],])

    
    X_out = np.concatenate(X_out,axis=0)
    y_out = np.concatenate(y_out,axis=0)
    #print(X_out.shape)
    #print(y_out)
    return X_out,y_out

In [ ]:
gammavec = 10**np.arange(-3,-1,0.1)
gammavec.shape = (gammavec.shape[0])
gammavec.shape
Cvec = np.arange(1,3.5,0.5)
Cvec.shape=(Cvec.shape[0])
Cvec.shape

In [ ]:
##### Gaussian kernel grid search
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, Normalizer


#gammavec = 10**np.arange(-2.25,-2.15,0.01)# got 0.00616595 #np.array(10**np.arange(-3,-0.5,0.1)) got 0.0063095734448
gammavec = 10**np.arange(0.,0.15,0.05) #0.3-1.5(0.25) got 1.995 #(-1,0.5,0.1) got 0.5
gammavec.shape = (gammavec.shape[0])
#Cvec = np.arange(3.15,3.55,0.10) #got 3.25 #np.arange(0.5,4,0.25) got 3.25
Cvec = np.arange(4.1,4.4,0.1) #3-5(0.25))got 4.25  #4-5, 4.0 (0.5)
Cvec.shape=(Cvec.shape[0])
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)
n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)

np.histogram(ya)

pip_gaussian.fit(Xa,ya.ravel())

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestgamma)
print(bestC)

In [ ]:
np.log10(1.1220184543)

In [ ]:
bestC = 4
bestgamma =10**0.4

In [ ]:
print(np.log10(bestgamma))
gammavec
print(10**np.arange(-2.25,-2.15,0.01))
#print(gammavec)
print(bestgamma)

In [ ]:
print(bestC)
Cvec
np.arange(3.15,3.55,0.10)

In [ ]:
########### Gaussian kernel methods

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer

num_items,num_features = Xtrain_1d.shape

ns = [300, 1000, 2000, 5000, 10000,20000, 40000, 60000]
score_gaussian = np.zeros((len(ns),1))
for i,n in enumerate(ns):
    #  gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)
    gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
    pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

#    gs_gaussian = SVC(kernel='rbf',gamma=0.0018)
#    pip_gaussian = make_pipeline(StandardScaler(),Normalizer(),gs_gaussian)

    Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n) 
    pip_gaussian.fit(Xa,ya)
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(score_gaussian[i])
    
score_gaussian

In [ ]:
print(bestC)
print(bestgamma)

In [ ]:
np.arange(-3,-0.5,0.1).shape
n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer

gammavec = np.array(10**np.arange(0.46,0.52,0.01)) #np.array(10**np.arange(-0.1,1,0.3))  best 0.5 #np.array(10**np.arange(-3,-0.5,0.1)) got 0.0063095734448
gammavec.shape = (6)
Cvec = np.arange(3.8,3.95,0.05)
#Cvec.append(1) #np.arange(3.15,3.55,0.10) #np.arange(0.5,4,0.25) got 3.25
Cvec.shape=(4)
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)
pip_gaussian.fit(Xa,ya)

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print(bestC)
print(bestgamma)
gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

pip_gaussian.fit(Xa,ya)
out=pip_gaussian.predict(Xtest_1d)
score = 1.0-accuracy_score(ytest, out)
print(score) # C=3, gamma = 3.16227766, score=0.0139
#C=3.9, gamma = 2.88403150313 score=0.0137

In [ ]:
n

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,Normalizer

n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)    

parameters = {'C':np.arange(0.75,5,0.25)}
gs_gaussian = GridSearchCV(SVC(kernel='linear'),param_grid=parameters)

pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)
pip_gaussian.fit(Xa,ya)

C_best = gs_gaussian.best_params_['C']

In [ ]:
C_best = gs_gaussian.best_params_['C']

In [ ]:
### linear

from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler,Normalizer

num_items,num_features = Xtrain_1d.shape

ns = [300, 1000, 2000, 5000, 10000]
score_gaussian = np.zeros((5,1))
for i,n in enumerate(ns):
    #  gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)
    gs_gaussian = SVC(kernel='linear',C=C_best)
    pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

#    gs_gaussian = SVC(kernel='rbf',gamma=0.0018)
#    pip_gaussian = make_pipeline(StandardScaler(),Normalizer(),gs_gaussian)

    Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)    
    pip_gaussian.fit(Xa,ya)
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(score_gaussian[i])
    
score_gaussian

In [ ]:
Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_linear.predict(Xe)
score = accuracy_score(ytest, predictedY)
score

In [ ]:
parameters = {"C":[1,1.1,1.11,1.05] ,
              "gamma":[0.1/num_features,1/num_features,10./num_features]}
gs_gaussian = GridSearchCV(SVC(kernel='rbf'),param_grid=parameters)

X = normalize(Xtrain_1d[0:n,:])
X = scale(X)

gs_gaussian.fit(X,ytrain[0:n])
gs_gaussian.best_params_

Xtest_1d = Xtest.reshape((len(Xtest),-1))
Xe = normalize(Xtest_1d)
Xe = scale(Xe)

predictedY = gs_gaussian.predict(Xe)
score = accuracy_score(ytest, predictedY)
score

In [ ]:
from sklearn.metrics import accuracy_score
print('mine:',np.sum(ytest==predictedY)/len(Xtest))
print('two:',accuracy_score(ytest, predictedY))

In [ ]:
#sampling the whole dataset 
n = 300
train_size=n/60000
test_size=0.5*n/60000

pipeline = make_pipeline(Normalizer(),StandardScaler(),gs)

# pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
cv = ShuffleSplit(n,n_iter=3,test_size=test_size, train_size=train_size)

scores = cross_val_score(pipeline,Xtrain_1d[0:n,:],ytrain[0:n],cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))

In [ ]:
gs.best_params_
pipeline.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit,cross_val_predict
from sklearn import metrics

def from_features_to_classif_scores(features,labels,ftest,ltest):
    #stack them for learning
    features = features.reshape((len(features),-1))
    ftest = ftest.reshape((len(ftest),-1))
    # apply pipeline
    n = len(features)
    
    pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=False))
    cv = ShuffleSplit(n,n_iter=3,test_size=0.1, train_size=0.9)
    
    scores = cross_val_score(pipeline,features,labels,cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % ((1.0-scores.mean())*100, scores.std() * 2*100))
    print('min score:',(1.0-scores.max())*100)
    return pipeline,scores

In [ ]:
from load_mnist_db import load_scattering_mnist
import time as time
import numpy as np
from keras.datasets import mnist

num_images = 60000
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_mnist(num_images = num_images, px=px,J=3,L=8)

num_points = [330, 1100, 2200, 5500, 11000, 22000, 60000]
for n in num_points:
    print('n=',n)
    pipeline, test_err = from_features_to_classif_scores(Xtrain[0:n,:,:,:],ytrain[0:n],Xtest[0:n,:,:,:],ytest[0:n])

In [ ]:
test_err

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.svm import SVC

X_train = Xtrain
y_train = ytrain
X_train = features.reshape((len(X_train),-1))
X_test = Xtest.reshape((len(Xtest),-1))

# apply pipeline
n = len(X_train)

pipeline = make_pipeline(Normalizer(),StandardScaler(),SVC(C=1.0, kernel='rbf', verbose=True))
#cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)

normalizer = Normalizer()
scaler = StandardScaler().fit(X_train)
X_train_normalized = normalizer.transform(X_train)
X_train_transformed = scaler.transform(X_train_normalized)

clf = SVC(kernel='rbf', C=1, verbose=True).fit(X_train_transformed, y_train)

X_test_normalized = normalizer.transform(X_test)
X_test_transformed = scaler.transform(X_test_normalized)

clf.score(Xtest, ytest)   


print('score:',scores)